### 不同类型的损失函数接口分别怎么用? 
下面三个接口都包含有损失函数：tf.keras.losses, tf.losses, tf.nn，它们的区别在于：    
+ （1）y_pred是位于softmax之前还是之后（即from_logits是否为True）：通过参数判断  
+ （2）y_true是标签值还是one_hot编码后的值：通过是否有sparse判断
+ （3）在本batch上计算的loss是平均值还是大小为batch size的向量：通过大小写判断
+ **注意：numpy或tensorflow在计算交叉熵cross entropy时，对数运算的底为自然数e=2.71828**  

In [1]:
import tensorflow as tf
import numpy as np
import os 

os.environ["CUDA_VISIBLE_DEVICES"] = '1'

+ **1.tf.keras.losses.BinaryCrossentropy是一个类，用于二分类任务中**  
+ **同名为：tf.losses.BinaryCrossentropy**  
-param:y_true：样本的标签     
-param:y_pred：经过softmax层后的输出，默认from_logits = False  
return:返回的是batch size个样本的平均loss    

In [2]:
# keras中二分类交叉熵损失函数的形式为loss = y_true * log(y_pred + EPSILON) + (1 - y_true) * log(1 - y_pred + EPSILON)，其中EPSILON=1e-7 
EPSILON = 1e-7
bce = tf.keras.losses.BinaryCrossentropy()         # 默认from_logits = False
y_true = np.array([0, 0, 1, 1])                    # 数据类型dtype也可以是浮点型
y_pred = np.array([1., 1., 1., 0.])
loss = bce(y_true, y_pred)
print('Loss: ', loss.numpy())

# 公式参考自：https://github.com/tensorflow/tensorflow/blob/1cf0898dd4331baf93fe77205550f2c2e6c90ee5/tensorflow/python/keras/backend.py
arr = y_true * np.log(y_pred + EPSILON) + (1 - y_true) * np.log(1 - y_pred + EPSILON)
loss2 = -np.mean(arr)
print("手动计算的值为：", loss2)

Loss:  11.568711280822754
手动计算的值为： 12.088571713218741


+ **2.tf.keras.losses.binary_crossentropy是一个函数，用于多分类任务中**   
+ **同名为：tf.keras.metrics.binary_crossentropy, tf.losses.binary_crossentropy, tf.metrics.binary_crossentropy**  
-param:y_true：样本的标签,如：[0, 1, 2]     
-param:y_pred：from_logits = True的输出 [[.9, .05, .05], [.05, .89, .06], [.05, .01, .94]]  
return:返回的是batch size个样本的平均loss  

In [3]:
# keras中二分类交叉熵损失函数的形式为loss = y_true * log(y_pred + EPSILON) + (1 - y_true) * log(1 - y_pred + EPSILON)，其中EPSILON=1e-7 
EPSILON = 1e-7        
y_true = np.array([0., 0., 1., 1.])
y_pred = np.array([1., 1., 1., 0.])
loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)    # 默认from_logits = False
print('Loss: ', loss.numpy())

# 公式参考自：https://github.com/tensorflow/tensorflow/blob/1cf0898dd4331baf93fe77205550f2c2e6c90ee5/tensorflow/python/keras/backend.py
arr = y_true * np.log(y_pred + EPSILON) + (1 - y_true) * np.log(1 - y_pred + EPSILON)
loss2 = -np.mean(arr)
print("手动计算的值为：", loss2)

Loss:  11.56871135293037
手动计算的值为： 12.088571713218741


+ **3. tf.keras.losses.CategoricalCrossentropy是一个类，用于多分类任务中**
+ **同名为：tf.losses.CategoricalCrossentropy**   
-param:y_true：one_hot编码后的标签  
-param:y_pred：经过softmax层后的输出，默认from_logits = False   
-return:返回的是batch size个样本的平均交叉熵loss，是一个标量  

In [4]:
# from_logits = False,构造对象时默认的方式。（即要求y_pred是经过softmax层后的值）
cce = tf.keras.losses.CategoricalCrossentropy()
y_true = [[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]]
y_pred = [[.9, .05, .05], [.05, .89, .06], [.05, .01, .94]]
loss = cce(y_true, y_pred)
print('Loss1: ', loss.numpy())
print("手动计算的值为1：",np.mean(-np.log([0.9,0.89,0.94])))


# from_logits = True，官网说这种数值更稳定。（即要求y_pred是softmax层之前的值）
cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
y_true = [[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]]
y_pred = [[.9, .05, .05], [.05, .89, .06], [.05, .01, .94]]
loss = cce(y_true, y_pred)
print('Loss2: ', loss.numpy())

arr = np.array([[.9, .05, .05], [.05, .89, .06], [.05, .01, .94]])
arr2 = np.exp(arr) / np.sum(np.exp(arr), axis=1)
arr3 = np.max(arr2, axis = 1)
print("手动计算的值为2：",np.mean(-np.log(arr3)))

Loss1:  0.09458993
手动计算的值为1： 0.09458991187728844
Loss2:  0.61106974
手动计算的值为2： 0.6110697851390151


+ **4.tf.keras.losses.categorical_crossentropy是一个函数**  
+ **同名为：tf.keras.metrics.categorical_crossentropy, tf.losses.categorical_crossentropy,tf.metrics.categorical_crossentropy**  
-param:y_true：one_hot编码后的标签  
-param:y_pred：经过softmax层后的输出，默认from_logits = False   
-return:返回的是batch size个样本的交叉熵loss向量，其长度等于batch size  

In [5]:
# from_logits = False,构造对象时默认的方式
y_true = [[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]]
y_pred = [[.9, .05, .05], [.05, .89, .06], [.05, .01, .94]]
loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
print('Loss: ', loss.numpy())
print('Loss的平均值: ', np.mean(loss.numpy()))
print("手动计算的值为：",np.mean(-np.log([0.9,0.89,0.94])))

Loss:  [0.10536055 0.11653383 0.06187541]
Loss的平均值:  0.09458993
手动计算的值为： 0.09458991187728844


+ **5.tf.keras.losses.SparseCategoricalCrossentropy是一个类，用于多分类任务中** 
+ **同名为：tf.losses.SparseCategoricalCrossentropy**  
-param:y_true：样本的标签,如：[0, 1, 2]     
-param:y_pred：from_logits = False的输出 [[.9, .05, .05], [.05, .89, .06], [.05, .01, .94]]  
return:返回的是batch size个样本的平均loss  

In [6]:
# y_true默认为数字标签，而不是one_hot编码
# 需要注意：y_true和y_pred必须是tf.Tensor类型的数据，否则会报错没有op属性
cce = tf.keras.losses.SparseCategoricalCrossentropy()
y_true = tf.constant([0., 1., 2.])
y_pred = tf.constant([[.9, .05, .05], [.05, .89, .06], [.05, .01, .94]])
loss = cce(y_true, y_pred)
print('Loss: ', loss.numpy())

print("手动计算的值为：",np.mean(-np.log([0.9,0.89,0.94])))

Loss:  0.09458992
手动计算的值为： 0.09458991187728844


+ **6.tf.keras.losses.sparse_categorical_crossentropy是一个函数，用于多分类任务中** 
+ **同名为：tf.keras.metrics.sparse_categorical_crossentropy, tf.losses.sparse_categorical_crossentropy, tf.metrics.sparse_categorical_crossentropy**  
-param:y_true：样本的标签,如：[0, 1, 2]     
-param:y_pred：from_logits = False的输出 [[.9, .05, .05], [.05, .89, .06], [.05, .01, .94]]  
return:返回的是batch size个样本的平均loss  

In [7]:
# y_true默认为数字标签，而不是one_hot编码
# 需要注意：y_true和y_pred必须是tf.Tensor类型的数据，否则会报错没有op属性
y_true = tf.constant([0., 1., 2.])
y_pred = tf.constant([[.9, .05, .05], [.05, .89, .06], [.05, .01, .94]])
loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
print('Loss: ', loss.numpy())
print('Loss的平均值: ', np.mean(loss.numpy()))

print("手动计算的值为：",np.mean(-np.log([0.9,0.89,0.94])))

Loss:  [0.10536056 0.11653379 0.0618754 ]
Loss的平均值:  0.09458992
手动计算的值为： 0.09458991187728844
